### Data

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn import metrics
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [ ]:
X_train = pd.read_pickle("x_train.pkl") # iid feature with target(label) for train dataset
X_test = pd.read_pickle("x_test.pkl") # iid feature with target(label) for test dataset
X_train_ = X_train.fillna(0)
X_test_ = X_test.fillna(0)

x_benign = X_train_[X_train_['label']==0]

scaler = MinMaxScaler()
x_benign = scaler.fit_transform(x_benign.iloc[:,:-3])
x_train = x_benign

x_test = scaler.transform(X_test_.iloc[:,:-3])
y_test = X_test_.iloc[:,-1]
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

### Model

#### Define Parameters

In [ ]:
# params
input_dim = 372 # this is the number of features we extracted. Should be adjusted according to your feature set
n_l1 = 1000
n_l2 = 1000
noise_dim = 50
z_dim = 500
batch_size = 100
n_epochs = 100
learning_rate = 0.001
beta1 = 0.9
results_path = './Results/'

# Placeholders for input data and the targets
x_input = tf.placeholder(dtype=tf.float32, shape=[None, input_dim], name='Input')
x_target = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Target')
noise_distribution = tf.placeholder(dtype=tf.float32, shape=[batch_size, noise_dim], name='Noise_distribution')

#### Define Tensorboard

In [ ]:
def form_results():
    """
    Forms folders for each run to store the tensorboard files, saved models and the log files.
    :return: three string pointing to tensorboard, saved models and log paths respectively.
    """
    folder_name = "/{0}_{1}_{2}_{3}_{4}_{5}_Adversarial_Autoencoder". \
        format(round(time.time()), z_dim, learning_rate, batch_size, n_epochs, beta1)
    tensorboard_path = results_path + folder_name + '/Tensorboard'
    saved_model_path = results_path + folder_name + '/Saved_models/'
    log_path = results_path + folder_name + '/log'
    if not os.path.exists(results_path + folder_name):
        os.mkdir(results_path + folder_name)
        os.mkdir(tensorboard_path)
        os.mkdir(saved_model_path)
        os.mkdir(log_path)
    return tensorboard_path, saved_model_path, log_path

#### Define component

In [ ]:
def dense(x, n1, n2, name):
    """
    Used to create a dense layer.
    :param x: input tensor to the dense layer
    :param n1: no. of input neurons
    :param n2: no. of output neurons
    :param name: name of the entire dense layer.i.e, variable scope name.
    :return: tensor with shape [batch_size, n2]
    """
    with tf.variable_scope(name, reuse=None):
        weights = tf.get_variable("weights", shape=[n1, n2],
                                  initializer=tf.random_normal_initializer(mean=0., stddev=0.01))
        bias = tf.get_variable("bias", shape=[n2], initializer=tf.constant_initializer(0.0))
        out = tf.add(tf.matmul(x, weights), bias, name='matmul')
        return out

In [ ]:
# The autoencoder network
def encoder(x, reuse=False):
    """
    Encode part of the autoencoder.
    :param x: input to the autoencoder
    :param reuse: True -> Reuse the encoder variables, False -> Create or search of variables before creating
    :return: tensor which is the hidden latent variable of the autoencoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Encoder'):
        e_dense_1 = tf.nn.relu(dense(x, input_dim, n_l1, 'e_dense_1'))
        e_dense_2 = tf.nn.relu(dense(e_dense_1, n_l1, n_l2, 'e_dense_2'))
        latent_variable = dense(e_dense_2, n_l2, z_dim, 'e_latent_variable')
        return latent_variable


def decoder(x, reuse=False):
    """
    Decoder part of the autoencoder.
    :param x: input to the decoder
    :param reuse: True -> Reuse the decoder variables, False -> Create or search of variables before creating
    :return: tensor which should ideally be the input given to the encoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Decoder'):
        d_dense_1 = tf.nn.relu(dense(x, z_dim, n_l2, 'd_dense_1'))
        d_dense_2 = tf.nn.relu(dense(d_dense_1, n_l2, n_l1, 'd_dense_2'))
        output = tf.nn.sigmoid(dense(d_dense_2, n_l1, input_dim, 'd_output'))
        return output


In [ ]:
# The GAN netowrk
def generator(x, reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Generator'):
        dc_den1 = tf.nn.relu(dense(x, noise_dim, n_l1, name='ge_den1'))
        dc_den2 = tf.nn.relu(dense(dc_den1, n_l1, n_l2, name='ge_den2'))
        output = dense(dc_den2, n_l2, z_dim, name='ge_output')
        return output


def discriminator(x, reuse=False):
    """
    Discriminator that is used to match the posterior distribution with a given prior distribution.
    :param x: tensor of shape [batch_size, z_dim]
    :param reuse: True -> Reuse the discriminator variables,
                  False -> Create or search of variables before creating
    :return: tensor of shape [batch_size, 1]
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Discriminator'):
        dc_den1 = tf.nn.relu(dense(x, z_dim, n_l1, name='dc_den1'))
        dc_den2 = tf.nn.relu(dense(dc_den1, n_l1, n_l2, name='dc_den2'))
        output = dense(dc_den2, n_l2, 1, name='dc_output')
        return output

#### Define Graph

In [ ]:
def train(x_train, x_test, y_test, train_model=True):
    """
    Used to train the autoencoder by passing in the necessary inputs.
    :param train_model: True -> Train the model, False -> Load the latest trained model and show the image grid.
    :return: does not return anything
    """
    with tf.variable_scope(tf.get_variable_scope()):
        encoder_output = encoder(x_input)
        decoder_output = decoder(encoder_output)

    with tf.variable_scope(tf.get_variable_scope()):
        fake_sample = generator(noise_distribution)
        d_fake = discriminator(fake_sample)
        d_real = discriminator(encoder_output, reuse=True)

    # Autoencoder loss
    autoencoder_loss = tf.reduce_mean(tf.square(x_target - decoder_output))
 
    # Discrimminator Loss
    dc_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_fake), logits=d_fake))
    dc_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(d_real), logits=d_real))
    dc_loss = dc_loss_fake + dc_loss_real

    # Generator loss
    generator_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_fake), logits=d_fake))

    all_variables = tf.trainable_variables()
    
    dc_var = [var for var in all_variables if 'dc_' in var.name]
    ge_var = [var for var in all_variables if 'ge_' in var.name] 
   

    # Optimizers
    autoencoder_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                                   beta1=beta1).minimize(autoencoder_loss)
    discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                                     beta1=beta1).minimize(dc_loss, var_list=dc_var)
    generator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                                 beta1=beta1).minimize(generator_loss, var_list=ge_var)

    init = tf.global_variables_initializer()

    # Tensorboard visualization
    tf.summary.scalar(name='Autoencoder Loss', tensor=autoencoder_loss)
    tf.summary.scalar(name='Discriminator Loss', tensor=dc_loss)
    tf.summary.scalar(name='Generator Loss', tensor=generator_loss)
    tf.summary.histogram(name='Encoder Distribution', values=encoder_output)
    tf.summary.histogram(name='Fake Sample Distribution', values=fake_sample)
    summary_op = tf.summary.merge_all()

    # Saving the model
    #saver = tf.train.Saver()
    step = 0
    with tf.Session() as sess:
        if train_model:
            tensorboard_path, saved_model_path, log_path = form_results()
            sess.run(init)
            writer = tf.summary.FileWriter(logdir=tensorboard_path, graph=sess.graph)
            for i in range(n_epochs):
                n_batches = int(x_train.shape[0] / batch_size)
                print("------------------Epoch {}/{}------------------".format(i, n_epochs))
                for b in range(1, n_batches + 1):
                    
                    noise_dist = np.random.randn(batch_size, noise_dim) * 5.
                    start, end = (b-1)*batch_size, b*batch_size
                    batch_x = x_train[start:end]
                    
                    sess.run(autoencoder_optimizer, feed_dict={x_input: batch_x, x_target: batch_x})
                    sess.run(discriminator_optimizer,
                             feed_dict={x_input: batch_x, x_target: batch_x, noise_distribution: noise_dist})
                    
                    sess.run(generator_optimizer, feed_dict={noise_distribution: noise_dist})
                    if b % 50 == 0:
                        a_loss, d_loss, g_loss, summary = sess.run(
                            [autoencoder_loss, dc_loss, generator_loss, summary_op],
                            feed_dict={x_input: batch_x, x_target: batch_x,
                                       noise_distribution: noise_dist})
                        writer.add_summary(summary, global_step=step)
                        print("Epoch: {}, iteration: {}".format(i, b))
                        print("Autoencoder Loss: {}".format(a_loss))
                        print("Discriminator Loss: {}".format(d_loss))
                        print("Generator Loss: {}".format(g_loss))
                        with open(log_path + '/log.txt', 'a') as log:
                            log.write("Epoch: {}, iteration: {}\n".format(i, b))
                            log.write("Autoencoder Loss: {}\n".format(a_loss))
                            log.write("Discriminator Loss: {}\n".format(d_loss))
                            log.write("Generator Loss: {}\n".format(g_loss))
                    step += 1

                #saver.save(sess, save_path=saved_model_path, global_step=step)
        else:
            # Get the latest results folder
            all_results = os.listdir(results_path)
            all_results.sort()
            saver.restore(sess, save_path=tf.train.latest_checkpoint(results_path + '/' + all_results[-1] + '/Saved_models/'))
            generate_image_grid(sess, op=decoder_image)

#### Train model

In [ ]:
train(x_train, x_test, y_test, train_model=True)

### trial

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np


def plot_result(log_path):
    #log_path = './Results/1566813215_1000_0.001_100_20_0.9_Adversarial_Autoencoder/log'
    with open(log_path + '/log.txt', 'r') as log:
        lines = log.readlines()
        index, ae_loss, d_loss, g_loss = [], [], [], []
        for line in lines:
            if line.startswith('E'):
                num = re.findall(r"\d+\.?\d*",line)
                index.append(f"{num[0]}-{num[1]}")
            if line.startswith('Aut'):
                val = re.findall(r"\d+\.?\d*",line)
                ae_loss.append(float(val[0]))
            if line.startswith('D'):
                val = re.findall(r"\d+\.?\d*",line)
                d_loss.append(float(val[0]))
            if line.startswith('G'):
                val = re.findall(r"\d+\.?\d*",line)
                g_loss.append(float(val[0]))  
        return index, ae_loss, d_loss, g_loss



In [ ]:

index1, ae_loss1, d_loss1, g_loss1 = plot_result('./Results/1566814923_1000_0.001_100_10_0.9_Adversarial_Autoencoder/log')

index2, ae_loss2, d_loss2, g_loss2 = plot_result('./Results/1566814590_1000_0.001_100_10_0.9_Adversarial_Autoencoder/log')

index3, ae_loss3, d_loss3, g_loss3 = plot_result('./Results/1566814170_1000_0.001_100_10_0.9_Adversarial_Autoencoder/log'
index4, ae_loss4, d_loss4, g_loss4 = plot_result('./Results/1566813900_1000_0.001_100_10_0.9_Adversarial_Autoencoder/log')     

In [ ]:
index, ae_loss, d_loss, g_loss, ks = plot_result('./Results/1566814170_1000_0.001_100_10_0.9_Adversarial_Autoencoder/log')

In [ ]:
import pandas as pd
df = pd.DataFrame([index, ae_loss, d_loss, g_loss, ks]).T
df.columns = ['index', 'ae_loss', 'd_loss', 'g_loss', 'ks']
df

### 比较不同修改对模型的影响

#### 比较autoencoder loss

In [ ]:
import pandas as pd
df_ae = pd.DataFrame([ae_loss1, ae_loss2, ae_loss3, ae_loss4]).T
df_ae.columns = ['iterate_encoder_and_ae_every_batch', 'iterate_encoder_and_ae_every_500_batches', 
                 'iterate_ae_not_encoder_every_500_batches',
                 'not_iterate_encoder']
df_ae

#### 比较d_loss, g_loss, ks

In [ ]:
df = pd.DataFrame([d_loss1, g_loss1, ks1, d_loss2, g_loss2, ks2, d_loss3,\
                     g_loss3, ks3, d_loss4, g_loss4, ks4]).T
df.columns = ['A_d_loss', 'A_g_loss', 'A_ks', 'B_d_loss', 'B_g_loss', 'B_ks', 'C_d_loss', 'C_g_loss', \
             'C_ks', 'D_d_loss', 'D_g_loss', 'D_ks']
print("A: ")
print("B: ")
print("C: ")
print("D: ")
df

#### 画一画d_loss, g_loss, ks的关系

In [ ]:
df_ = df[['A_d_loss', 'A_g_loss', 'A_ks']]
df_ = df_[df_['A_d_loss']<10]
df_ = df_[df_['A_g_loss']<10]
df_good = df_[df_['A_ks']>0.1]
df_bad = df_[df_['A_ks']<0.1]

In [ ]:
df_['square'] = df_['A_d_loss']**2 + df_['A_g_loss']**2

d_loss, g_loss does not reflect the performance of gan

decrease of loss does not bring ks improvement

In [ ]:
df_.sort_values('square')

In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(15,15))
ax = plt.subplot(111, projection='3d') 

ax.scatter(df_good['A_d_loss'], df_good['A_g_loss'], df_good['A_ks'], c='b')  
ax.scatter(df_bad['A_d_loss'], df_bad['A_g_loss'], df_bad['A_ks'], c='r')


ax.set_zlabel('Z') 
ax.set_ylabel('Y')
ax.set_xlabel('X')
plt.show()
